In [6]:
import dgl
import dgl.function as fn
import dgl.nn as gnn

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from utils import create_heterograph

Цель: предсказать, является ли связь между пользователем и товаром связью типа `click` или `dislike`.

Решение: 
1. Для получения представлений узлов берем любую модель, работающую с гетерографом (тут `RGCN` из `03_dgl_node_graph_classification`)
2. Получаем специальное представление графа, в котором сохранены типы узлов и есть только 1 тип ребер между пользователем и товаров (т.е. все типы узлов между этими типами узлов объединены)
3. Прогоняем его через `HeteroDotProductPredictor` из `08_dgl_edge_regression` (если многоклассовая классификация, то можно реализовать аналог `MLPPredictor`)

In [12]:
# пример с получением нужного представления графа
data = {('user', 'click', 'item'): ([0, 0, 1], [0, 2, 1]),
        ('user', 'dislike', 'item'): ([0, 1], [1, 0]),
        ('item', 'clicked-by', 'user'): ([0, 2, 1], [0, 0, 1]),
        }
G = dgl.heterograph(data)
H = G['user', :, 'item']
print(f'{G=}')
print()
print(f'{H=}')
print()
print(f'{H.edges()=}')

G=Graph(num_nodes={'item': 3, 'user': 2},
      num_edges={('item', 'clicked-by', 'user'): 3, ('user', 'click', 'item'): 3, ('user', 'dislike', 'item'): 2},
      metagraph=[('item', 'user', 'clicked-by'), ('user', 'item', 'click'), ('user', 'item', 'dislike')])

H=Graph(num_nodes={'user': 2, 'item': 3},
      num_edges={('user', 'click+dislike', 'item'): 5},
      metagraph=[('user', 'item', 'click+dislike')])

H.edges()=(tensor([0, 0, 1, 0, 1]), tensor([0, 2, 1, 1, 0]))


In [14]:
class RGCN(nn.Module):
    def __init__(self, n_inputs, n_hidden, rel_names):
        super().__init__()
        # HeteroGraphConv использует различные подмодули для подграфов на 
        # основе соответствующих отношений
        # отношение определяется тройкой (src_T, rel_T, dst_T)
        # если для каких-то отношений используются одинаковые dst_T,
        # то результаты для них будут сагрегированы указанным методом aggregate
        conv1_modules = {rel: gnn.GraphConv(n_inputs, n_hidden) for rel in rel_names}
        conv2_modules = {rel: gnn.GraphConv(n_hidden, n_hidden) for rel in rel_names}
        self.conv1 = gnn.HeteroGraphConv(conv1_modules, aggregate='sum')
        self.conv2 = gnn.HeteroGraphConv(conv2_modules, aggregate='sum')

    def forward(self, G, features):
        # HeteroGraphConv принимает на вход словарь тип отношения: фичи узлов и 
        # возвращает словарь такой же структуры
        out = self.conv1(G, features)
        out = {k: F.relu(v) for k, v in out.items()}
        out = self.conv2(G, out)
        return out

class HeteroMLPPredictor(nn.Module):
    def __init__(self, n_node_features, n_classes):
        super().__init__()
        self.linear = nn.Linear(2 * n_node_features, n_classes)

    def gen_edge_feature(self, edges):
        src = edges.src['h']
        dst = edges.dst['h']
        src_dst = torch.cat([src, dst], dim=1)
        edge_feature = self.linear(src_dst)
        return {'e_h': edge_feature}

    def forward(self, G, features, etype):
        # features - это представления узлов для всех типов
        with G.local_scope():
            # таким образом можно присвоить свойство h всем типам узлов сразу
            G.ndata['h'] = features
            # в данной задаче получается так, что в графе остается 1 тип ребер
            # т.е. аргумент etype можно было бы опустить
            G.apply_edges(self.gen_edge_feature, etype=etype)
            return G.edges[etype].data['e_h']
            

In [55]:
class GCN(nn.Module):
    def __init__(self, n_inputs, n_hidden, n_outputs, rel_names):
        super().__init__()
        self.conv = RGCN(n_inputs, n_hidden, n_hidden, rel_names)
        self.predictor = HeteroMLPPredictor(n_hidden, n_outputs)

    def forward(self, G, features, H):
        # прогоняем "обычный" гетерограф через conv, получаем представления узлов
        # c разбивкой по типам
        out = self.conv(G, features)
        # H - это "упрощенная" версия G, где ребра разных типов
        # между пользователем и товаром слиты в один тип
        assert len(H.etypes) == 1
        # для каждого ребра (#ребер типа click + #ребер типа dislike)
        # получаем вектор предсказания
        out = self.predictor(H, out, H.etypes[0])
        return out

In [60]:
G = create_heterograph()
features = {ntype: G.nodes[ntype].data['feature'] for ntype in G.ntypes}
edge_labels = H.edata[dgl.ETYPE]
# в edge_labels лежит подмножество G.etypes
# перенумеруем их, начиная с 0
renum_map = {label.item(): idx for idx, label in enumerate(edge_labels.unique())}
edge_labels = torch.LongTensor([renum_map[label.item()] for label in edge_labels])
n_classes = len(renum_map)

H = G['user', :, 'item']

model = GCN(10, 20, n_classes, G.etypes)
optimizer = optim.Adam(model.parameters(), lr=.01)
criterion = nn.CrossEntropyLoss()

for epoch in range(101):
    # forward
    logits = model(G, features, H)
    loss = criterion(logits, edge_labels)
    # backward
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if not epoch % 5:
        print(f'Epoch #{epoch} loss={loss.item()}')


Epoch #0 loss=0.9480441212654114
Epoch #5 loss=0.38045939803123474
Epoch #10 loss=0.2922082841396332
Epoch #15 loss=0.26186424493789673
Epoch #20 loss=0.24391502141952515
Epoch #25 loss=0.22541563212871552
Epoch #30 loss=0.209209606051445
Epoch #35 loss=0.19780096411705017
Epoch #40 loss=0.19008786976337433
Epoch #45 loss=0.1848503053188324
Epoch #50 loss=0.18062101304531097
Epoch #55 loss=0.17699414491653442
Epoch #60 loss=0.17399506270885468
Epoch #65 loss=0.17157797515392303
Epoch #70 loss=0.1696457415819168
Epoch #75 loss=0.16809231042861938
Epoch #80 loss=0.166630819439888
Epoch #85 loss=0.16543890535831451
Epoch #90 loss=0.16435131430625916
Epoch #95 loss=0.1634066253900528
Epoch #100 loss=0.16251473128795624
